In [273]:
import random
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from scipy.interpolate import interp1d
import torch.nn.init as init

## Custom Function Activation -- Module

In [274]:
class CustomFunction(nn.Module):
    def __init__(self,function):
        super().__init__()
        self.function = function
        
    def forward(self, x):
        global lim_inf,lim_sup
        x = torch.clamp(x, lim_inf, lim_sup)
        x = x.detach().numpy()
        return torch.tensor(self.function(x), dtype=torch.float32)

## Class MLP_CUSTOM

In [275]:
class MLP_Custom(nn.Module):
    def __init__(self,input_size,hidden_size,output_size,genome):
        super().__init__()
        self.fc1 = nn.Linear(input_size,hidden_size)
        self.custom_activation = CustomFunction(self.act_func_generator(genome))
        self.fc2 = nn.Linear(hidden_size,output_size)
        
        global seed
        if seed is not None:
            torch.manual_seed(seed)
    def forward(self,x):
        x=self.fc1(x)
        x=self.custom_activation(x)
        x=self.fc2(x)
        return x
    
    def act_func_generator(self,genome):
     global lim_inf,lim_sup
     x_dom = np.linspace(lim_inf,lim_sup, len(genome))
     function = interp1d(x_dom, genome, kind='cubic', fill_value="extrapolate")
     return function
    
    def init_weights(self):
     init.xavier_uniform_(self.fc1.weight)
     init.zeros_(self.fc1.bias)
     init.xavier_uniform_(self.fc2.weight)
     init.zeros_(self.fc2.bias)


In [276]:
 def print_par(index):
  for name, param in net_pop[index].network.named_parameters():
   print(f'Ind:{index},{name}: {np.round(np.array(param.data),3).reshape(1, -1) }')

In [277]:
class Individual(object):
    def __init__(self,genome,input_size,hidden_size,output_size):
        self.genome  = genome
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.score= None       
        self.network = MLP_Custom(self.input_size,self.hidden_size,self.output_size,self.genome)
        self.init_pars=self.network.parameters()
        self.trained_pars= None
    
    # Create chromosome
    @classmethod
    def create_chromosome(self,level_inf,level_sup,step,dna_length,inter):  
     if inter == False:
      dna = np.random.choice(np.arange(level_inf, level_sup, step), size=dna_length, replace=True)
     else:
      dna = np.random.randint(level_inf, level_sup, dna_length)
     return dna
    
    def training(self,lr,epochs,X,Y):
     mse_loss = nn.MSELoss()
     optimizer = optim.Adam(self.network.parameters(), lr)
    
     loss_values_custom = []
     for epoch in range(epochs):
        self.network.train()
        optimizer.zero_grad()
        output = self.network(X)
        loss = mse_loss(output, Y)
        #print(loss.item())
        loss_values_custom.append(loss.item())
        loss.backward()
        optimizer.step()
     self.score = loss_values_custom[-1] 
     self.trained_pars=self.network.parameters()

In [278]:
# ----------------Activation Parameters-------------------
# Codomain limits
level_inf=0
level_sup=3
inter = False
step=0.01
# Domain limits
lim_inf=-3
lim_sup=3
# DNA Length
dna_length=20
# ----------------Training Parameters-------------------
# input_size
input_size=1
# hidden_size
hidden_size=10
# output_size
output_size=1
# Epochs
epochs=1000
# Learning Rate
lr=0.01
# Seed parameters generation
seed = 42
# ----------------Genetic Algorithm Parameters-------------------
# Number of networks
pop_size = 10
# Number of generations
N_gens=100

# ----------------Initilization----------------------------------
net_pop=[]
for i in range(pop_size):
    # Generation DNA for activation function
    genome = Individual.create_chromosome(level_inf,level_sup,step,dna_length,inter)
    # Networks Initialization
    net_pop.append(Individual(genome,input_size,hidden_size,output_size))

In [288]:
 print_par(3)

Ind:3,fc1.weight: [[ 0.765  0.83  -0.234  0.919 -0.219  0.202 -0.487  0.587  0.882 -0.734]]
Ind:3,fc1.bias: [[ 0.869  0.187  0.739  0.135  0.482 -0.141  0.771  0.148 -0.467  0.255]]
Ind:3,fc2.weight: [[-0.166 -0.609  0.711  0.245  0.05   0.029 -0.203  0.055 -0.31  -0.312]]
Ind:3,fc2.bias: [[0.733]]


In [280]:
X=np.linspace(-2,2,1000).reshape(-1,1)
Y=np.sin(X)
X_tensor = torch.tensor(X,dtype=torch.float32)
Y_tensor = torch.tensor(Y,dtype=torch.float32)

for i in range(pop_size):
 #print(f'Starting training of network:{i}')
 net_pop[i].training(lr,epochs,X_tensor,Y_tensor)

In [281]:
 print_par(2)

Ind:2,fc1.weight: [[ 0.765  0.83  -0.234  0.919 -0.219  0.202 -0.487  0.587  0.882 -0.734]]
Ind:2,fc1.bias: [[ 0.869  0.187  0.739  0.135  0.482 -0.141  0.771  0.148 -0.467  0.255]]
Ind:2,fc2.weight: [[-0.255 -0.069  0.395 -0.177  0.27   0.531 -0.007 -0.121 -0.193 -0.13 ]]
Ind:2,fc2.bias: [[-0.184]]


In [282]:
score_list=[]
for i,ind in enumerate(net_pop):
    score_list.append(ind.score)

score_list.index(min(score_list))


9

In [283]:
subset=random.sample(net_pop,5)

In [284]:
subset[0]

In [285]:
# while not solution_found:
#     population = sorted(population, key=lambda x: x.fitness)
#     if population[0].fitness <= 0:
#         solution_found=True
#         break
#     new_generation=[]
#     
#     #Selection
#     x = int((15*pop_size)/100)
#     new_generation.extend(population[:x])
#     
#     #Crossover & mutation
#     x = int((85*pop_size)/100)
#     for _ in range(x):
#         first_parent = random.choice(population[:50])
#         second_parent = random.choice(population[:50])
#         child = first_parent.crossover(second_parent)
#         new_generation.append(child)
#     
#     population=new_generation
#     generation += 1

In [286]:
    # # Mutuation
    # @classmethod
    # def genes_mutated(self):
    #     global genes
    #     random_gene = random.choice(genes)
    #     return random_gene
    # 
    # def fitness_score(self):
    #     global Solution
    #     fitness = 0
    #     for xx,yy in zip(self.chromosome,Solution):
    #         if xx!= yy: fitness+=1
    #     return fitness

In [287]:
    # def crossover(self,second_parent): #Istanze first and second parent chiamata sull'istanza stessa
    #     # Nuovo cromosoma del figlio
    #     child_chromosome = []
    #     # Con una determinata probabilità viene scelto un gene da un genitore
    #     for genome_first_parent,genome_second_parent in zip(self.chromosome,second_parent.chromosome):
    #      prob=random.random()
    #      if prob<0.20:
    #          child_chromosome.append(genome_first_parent)
    #      elif prob<0.90:
    #          child_chromosome.append(genome_second_parent)
    #      else:
    #      # Mutazione
    #         child_chromosome.append(self.genes_mutated())
    #         
    #     return Chromosome(child_chromosome)